In [1]:
import numpy as np
import arcode
import codec
import struct
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
from PIL import Image

In [2]:
def show_data(x_in, pre_in):
    mean = np.mean(x_in)
    std = np.std(x_in)
    Max = np.max(x_in)
    Min = np.min(x_in)
    dtype = x_in.dtype
    
    print(pre_in + ' shape: ' + str(x_in.shape) + ' , mean = ' + str(mean) 
          + ' , std = ' + str(std) + ' , Max = ' + str(Max) + ' , Min = ' + str(Min) 
          + ' , type=' + str(dtype))

In [3]:
def oc_fname(i_in):
    rname = '/f00' + str(i_in) + '.png'
    if i_in >9: rname = '/f0' + str(i_in) + '.png'
    if i_in >99: rname = '/f' + str(i_in) + '.png'
        
    return rname

In [4]:
def read_COMPINT(prefC, cname):
    o4=np.zeros([1, nx, ny, ncomp])
    for k in range(ncomp):
        inname = prefC + str(k) + cname
        #print("  read_COMPINT: " + inname)
        tim=Image.open(inname)
        tima=np.array(tim)
        o4[0,:,:,k] = tima
                
    return o4

In [5]:
def ar_code(x_in, oname):
    img = x_in.flatten()
    olen = len(img)
    img = img.reshape(olen,1,1)
    
    bits = codec.bitstream(dat=img)
    bits.h = img.shape[0]
    bits.w = img.shape[1]
    bits.c = img.shape[2]
    entro_coder = codec.entropy_codec(Mode='Arith')
    entro_coder.encode(bits, oname)
    
    size1= os.path.getsize(oname + 'dat')
    size2= os.path.getsize(oname + 'arith')

    #olen, size1, size2 in unit of Byte (assuming input is 8bit int)
    return olen, size1, size2

In [6]:
def ar_decode(aname, dname):
    ar = arcode.ArithmeticCode(False)
    ar.decode_file(aname, dname)

    file_bin = open(dname,'rb')
    s = file_bin.read()
    H,W,C = struct.unpack('3i',s[:12])
    #print('H= ' + str(H) + ' ; W= ' + str(W) + ' ; C= ' + str(C))
    
    bytes=[]
    for i in range(H*W*C):
        bytes.append(struct.unpack('b',s[12+i:12+i+1]))
    file_bin.close()
    bin_data = np.array(bytes).reshape(H,W,C)
    bin_data = bin_data.flatten()
    
    return bin_data

In [68]:
'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_884/COMPINT'
nx=256
ny=384
ncomp=4
'''

'''
pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Bc_16164/COMPINT'
nx=128
ny=192
ncomp=4
'''


pref = '/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Gc_32324/COMPINT'
nx=64
ny=96
ncomp=4


print(pref)

/home/saberbud/Programs/xy_TF_sectioned_compression/Kodak_Gc_32324/COMPINT


In [69]:
i_f = 7
t_C = read_COMPINT(pref, oc_fname(i_f))

show_data(t_C, 't_C')
for i in range(4):
    show_data(t_C[...,i], 't_C_'+str(i))

t_C shape: (1, 64, 96, 4) , mean = 129.6107177734375 , std = 36.6196228104246 , Max = 218.0 , Min = 36.0 , type=float64
t_C_0 shape: (1, 64, 96) , mean = 94.89290364583333 , std = 14.488577187308314 , Max = 148.0 , Min = 36.0 , type=float64
t_C_1 shape: (1, 64, 96) , mean = 168.82552083333334 , std = 15.12451014150208 , Max = 218.0 , Min = 117.0 , type=float64
t_C_2 shape: (1, 64, 96) , mean = 97.32242838541667 , std = 12.624795464053834 , Max = 162.0 , Min = 50.0 , type=float64
t_C_3 shape: (1, 64, 96) , mean = 157.40201822916666 , std = 14.419242463403469 , Max = 218.0 , Min = 100.0 , type=float64


In [70]:
idc = 3
data_t = t_C[0,:,:,idc]
data_t = data_t - 128.0
data_t = data_t.astype(np.int8)
show_data(data_t, 'data_t')

data_t shape: (64, 96) , mean = 29.402018229166668 , std = 14.419242463403469 , Max = 90 , Min = -28 , type=int8


In [71]:
oname = 'fAc'
olen, size1, size2 = ar_code(data_t, oname)

print('olen' + str(olen))
print('size1' + str(size1))
print('size2' + str(size2))

ratio1 = float(size1)/float(olen)
ratio2 = float(size2)/float(olen)

print('ratio1= ' + str(ratio1))
print('ratio2= ' + str(ratio2))

olen6144
size16156
size24445
ratio1= 1.001953125
ratio2= 0.723470052083


In [72]:
aname = oname + 'arith'
dname = 'dAc.dat'
dcode = ar_decode(aname, dname)

dlen = len(dcode)
print('dlen= ' + str(dlen))

mse = np.mean(np.square(dcode-data_t.flatten()))
print('mse = ' + str(mse))

dlen= 6144
mse = 0.0


In [73]:
fs = 1; fe = 25
nf = fe-fs
size1s = np.zeros([nf,4])
size2s = np.zeros([nf,4])
olens = np.zeros([nf,4])
d_lens = np.ones([nf,4])
mses = np.zeros([nf,4])

for i in range(fs,fe):
    print(i)
    t_C = read_COMPINT(pref, oc_fname(i))
    show_data(t_C, '  t_C')
    
    for idc in range(4):
        data_t = t_C[0,:,:,idc]
        data_t = data_t - 128.0
        data_t = data_t.astype(np.int8)
        show_data(data_t, ' channel'+str(idc))
        olen, size1, size2 = ar_code(data_t, oname)

        print('   olen: ' + str(olen) + ' Bytes')
        print('   size1: ' + str(size1) + ' Bytes')
        print('   size2: ' + str(size2) + ' Bytes')

        ratio1 = float(size1)/float(olen)
        ratio2 = float(size2)/float(olen)

        print('ratio1= ' + str(ratio1))
        print('ratio2= ' + str(ratio2))
        
        dcode = ar_decode(aname, dname)
        dlen = len(dcode)
        print('dlen= ' + str(dlen))

        idx = i-fs
        
        if dlen == olen:
            d_lens[idx,idc] = 0
            mse = np.mean(np.square(dcode-data_t.flatten()))
            mses[idx,idc] = mse
            print('mse = ' + str(mse))
        if dlen != olen:
            d_lens[idx,idc] = 1
            mses[idx,idc] = 99.99
            print('!Length differ!')
        
        size1s[idx,idc] = size1
        size2s[idx,idc] = size2
        olens[idx,idc] = olen
        print('')
    
    print('\n')

1
  t_C shape: (1, 64, 96, 4) , mean = 129.51904296875 , std = 36.320695414049176 , Max = 224.0 , Min = 29.0 , type=float64
 channel0 shape: (64, 96) , mean = -31.820638020833332 , std = 16.475539906720186 , Max = 27 , Min = -87 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4730 Bytes
ratio1= 1.001953125
ratio2= 0.769856770833
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 39.193359375 , std = 17.05075473848884 , Max = 96 , Min = -10 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4719 Bytes
ratio1= 1.001953125
ratio2= 0.76806640625
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -29.8388671875 , std = 15.869731729776316 , Max = 46 , Min = -99 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4610 Bytes
ratio1= 1.001953125
ratio2= 0.750325520833
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = 28.542317708333332 , std = 14.774548105155752 , Max = 92 , Min = -28 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
 

dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -32.779134114583336 , std = 20.31692774255192 , Max = 54 , Min = -102 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4995 Bytes
ratio1= 1.001953125
ratio2= 0.81298828125
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = 30.83935546875 , std = 17.767708870049326 , Max = 99 , Min = -43 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4833 Bytes
ratio1= 1.001953125
ratio2= 0.78662109375
dlen= 6144
mse = 0.0



9
  t_C shape: (1, 64, 96, 4) , mean = 129.91748046875 , std = 42.67981866170137 , Max = 231.0 , Min = 22.0 , type=float64
 channel0 shape: (64, 96) , mean = -40.051920572916664 , std = 12.146890391420815 , Max = 45 , Min = -92 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4277 Bytes
ratio1= 1.001953125
ratio2= 0.696126302083
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 49.665364583333336 , std = 13.212493530130704 , Max = 103 , Min = -15 , type=int8
   olen: 61

dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 39.924967447916664 , std = 14.595581222064364 , Max = 86 , Min = 0 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4656 Bytes
ratio1= 1.001953125
ratio2= 0.7578125
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -30.227376302083332 , std = 11.237560797321672 , Max = 7 , Min = -82 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4373 Bytes
ratio1= 1.001953125
ratio2= 0.711751302083
dlen= 6144
mse = 0.0

 channel3 shape: (64, 96) , mean = 28.676432291666668 , std = 11.915861355973835 , Max = 75 , Min = -9 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4442 Bytes
ratio1= 1.001953125
ratio2= 0.722981770833
dlen= 6144
mse = 0.0



17
  t_C shape: (1, 64, 96, 4) , mean = 129.249755859375 , std = 28.87401285073583 , Max = 228.0 , Min = 45.0 , type=float64
 channel0 shape: (64, 96) , mean = -22.393880208333332 , std = 16.532740608481 , Max = 23 , Min = -77 , type=int8
   olen: 6144 By

   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4591 Bytes
ratio1= 1.001953125
ratio2= 0.747233072917
dlen= 6144
mse = 0.0



24
  t_C shape: (1, 64, 96, 4) , mean = 129.4619140625 , std = 36.75834542407386 , Max = 231.0 , Min = 30.0 , type=float64
 channel0 shape: (64, 96) , mean = -32.4072265625 , std = 15.436852208598998 , Max = 23 , Min = -98 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4652 Bytes
ratio1= 1.001953125
ratio2= 0.757161458333
dlen= 6144
mse = 0.0

 channel1 shape: (64, 96) , mean = 40.644694010416664 , std = 17.805203947208323 , Max = 103 , Min = -3 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4780 Bytes
ratio1= 1.001953125
ratio2= 0.777994791667
dlen= 6144
mse = 0.0

 channel2 shape: (64, 96) , mean = -30.629720052083332 , std = 14.770359211940452 , Max = 15 , Min = -98 , type=int8
   olen: 6144 Bytes
   size1: 6156 Bytes
   size2: 4636 Bytes
ratio1= 1.001953125
ratio2= 0.754557291667
dlen= 6144
mse = 0.0

 channel3 shape: (64, 

In [74]:
for idx in range(nf):
    print(idx+1)
    
    ratio_1 = np.mean(size1s[idx,:])/np.mean(olens[idx,:])
    ratio_2 = np.mean(size2s[idx,:])/np.mean(olens[idx,:])

    #print('   size_orig= ' + str(size_orig))
    print('   ratio_1= ' + str(ratio_1))
    print('   ratio_2= ' + str(ratio_2))
    print(' ')

1
   ratio_1= 1.001953125
   ratio_2= 0.7596842447916666
 
2
   ratio_1= 1.001953125
   ratio_2= 0.6405029296875
 
3
   ratio_1= 1.001953125
   ratio_2= 0.732421875
 
4
   ratio_1= 1.001953125
   ratio_2= 0.7684326171875
 
5
   ratio_1= 1.001953125
   ratio_2= 0.7879231770833334
 
6
   ratio_1= 1.001953125
   ratio_2= 0.7647298177083334
 
7
   ratio_1= 1.001953125
   ratio_2= 0.7262369791666666
 
8
   ratio_1= 1.001953125
   ratio_2= 0.7989095052083334
 
9
   ratio_1= 1.001953125
   ratio_2= 0.6949462890625
 
10
   ratio_1= 1.001953125
   ratio_2= 0.7116292317708334
 
11
   ratio_1= 1.001953125
   ratio_2= 0.7204182942708334
 
12
   ratio_1= 1.001953125
   ratio_2= 0.6701253255208334
 
13
   ratio_1= 1.001953125
   ratio_2= 0.765869140625
 
14
   ratio_1= 1.001953125
   ratio_2= 0.7816569010416666
 
15
   ratio_1= 1.001953125
   ratio_2= 0.7539876302083334
 
16
   ratio_1= 1.001953125
   ratio_2= 0.7322998046875
 
17
   ratio_1= 1.001953125
   ratio_2= 0.7611490885416666
 
18
   ratio_